In [1]:
from sepal_ui import widgetFactory as wf
from sepal_ui import widgetBinding as wb

import ipyvuetify as v
from utils import messages as ms

In [2]:
#input and output as mutable object 
class Bands_input_io:
    def __init__(self):        
        #output 
        self.forest_bands = None
        self.non_forest_bands = None
        self.validate = False

bi_io = Bands_input_io()

In [3]:
#debug 
values = [1, 2, 3, 4, 5, 6]

#create the output 
bi_output = wf.OutputWidget(ms.BANDS_INTRO)

#create the button 
bi_btn = wf.ProcessBtn(ms.BANDS_BTN)

#create the inputs 
bi_select_f = v.Select(
    items=values,
    multiple=True,
    label='forest bands',
    v_model=None
)
bi_select_nf = v.Select(
    multiple=True,
    items=values,
    label='non-forest bands',
    v_model=None
)

#bind the widgets
wb.bind(bi_select_f, bi_io, 'forest_bands', bi_output)
wb.bind(bi_select_nf, bi_io, 'non_forest_bands', bi_output)

#create a bands selector tile
id_ = "input_widget"
title = "Select bands"

bi_tile = wf.Tile(
    id_,
    title,
    btn=bi_btn,
    inputs=[bi_select_f,bi_select_nf],
    output=bi_output
)

In [4]:
#function to reset the bands 
def reset_bands(values, f = bi_select_f, nf = bi_select_nf):
    
    #reset the output
    setattr(bi_io, 'forest_bands', None)
    setattr(bi_io, 'forest', None)
    
    #reset the select 
    f.v_model = None
    nf.v_model = None
    
    #add the new values 
    f.items = values
    nf.items = values
    
    return

In [5]:
#validate the band selection 
from sepal_ui.scripts import utils as su
from functools import partial

def process_start(widget, event, data, output):
    
    #toggle the loading button 
    su.toggleLoading(widget)
    
    #load inputs 
    forest = getattr(bi_io, 'forest_bands')
    nforest = getattr(bi_io, 'non_forest_bands')
    
    #check input 
    if not wb.checkInput(forest, output, ms.NO_BANDS): return su.toggleLoading(widget)
    if not wb.checkInput(nforest, output, ms.NO_BANDS): return su.toggleLoading(widget)
    
    #check if overwrite 
    for val in forest:
        if val in nforest:
            su.displayIO(output, ms.BANDS_OVERRIDE, 'error')
            su.toggleLoading(widget)
            return 
    
    setattr(bi_io, 'validate', True)
    
    su.displayIO(output, ms.BANDS_VALID, 'success')
    
    #toggle the loading button 
    su.toggleLoading(widget)
    
    return 

bi_btn.on_event('click', partial(
    process_start,
    output= bi_output
))
    

In [7]:
bi_tile

Layout(align_center=True, children=[Card(children=[Html(children=['Select bands'], tag='h2'), Flex(children=[L…

In [8]:
bi_io.forest_bands

[1, 2]